# Setting Up Google Drive

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 12.0MB/s 


In [0]:
!pip install tensorlayer

    100% |████████████████████████████████| 317kB 23.8MB/s 
    100% |████████████████████████████████| 61kB 22.3MB/s 
    100% |████████████████████████████████| 25.3MB 1.4MB/s 
    100% |████████████████████████████████| 706kB 24.5MB/s 
spacy 2.0.18 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
scikit-image 0.14.2 has requirement pillow>=4.3.0, but you'll have pillow 4.1.1 which is incompatible.
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
google-colab 1.0.0 has requirement requests~=2.18.0, but you'll have requests 2.20.1 which is incompatible.
featuretools 0.4.1 has requirement pandas>=0.23.0, but you'll have pandas 0.22.0 which is incompatible.
fastai 1.0.48 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Found existing installation: requests 2.18.4


# **Importing DIV2K dataset and unzipping it**

In [0]:
file_id = '1Ri1RBc20RAvSQrjxX_NA0236IPir8dDO'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('DIV2K_train_LR_bicubic_X4.zip')

file_id = '1YBLqyzUbLwD7zj5p7Obyb8Fv1P9Nb8j4'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('DIV2K_train_HR.zip')

file_id = '1AP6x1_8VszT0GqEaGGlQvYq1oy1VIMf7'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('DIV2K_valid_LR_bicubic_X4.zip')

file_id = '13JZ1-hXbw18PWpL8ThKzYaEPwxVOXR78'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('DIV2K_valid_HR.zip')

#downloading vgg19.npy
file_id = '1czDkZN48B2kLk-SWs6lBGyEt-OrHQ207'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('vgg19.npy')

#downloading model.py SRGAN_g & SRGAN_d
file_id = '1jGx597P3wqEsILwT622g1dmKuG1hTLz7'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model.py')

##downloading temp d_train.npz from drive
file_id = '1uYqJwPgb98rZbwbqIExnSBy1VnqlqhoN'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('checkpoint/d_train.npz')

In [0]:
#during eval -- for model.py from gdrive
file_id = '1jGx597P3wqEsILwT622g1dmKuG1hTLz7'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('model.py')

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('DIV2K_train_HR.zip', 'r')
zip_ref.extractall('DIV2K_train_HR')

zip_ref = zipfile.ZipFile('DIV2K_valid_LR_bicubic_X4.zip', 'r')
zip_ref.extractall('DIV2K_valid_LR_bicubic_X4')

zip_ref = zipfile.ZipFile('DIV2K_valid_HR.zip', 'r')
zip_ref.extractall('DIV2K_valid_HR')

zip_ref = zipfile.ZipFile('DIV2K_train_LR_bicubic_X4.zip', 'r')
zip_ref.extractall('DIV2K_train_LR_bicubic_X4')
zip_ref.close()

# **Actual starts from here**

In [0]:
##config dictionary and config.TRAIN dictionary for learning parameters
from easydict import EasyDict as edict
import json

config = edict()
config.TRAIN = edict()

## Adam
config.TRAIN.batch_size = 16
config.TRAIN.lr_init = 1e-4
config.TRAIN.beta1 = 0.9

## initialize G
config.TRAIN.n_epoch_init = 100

## adversarial learning (SRGAN)
config.TRAIN.n_epoch = 100
config.TRAIN.lr_decay = 0.1
config.TRAIN.decay_every = int(config.TRAIN.n_epoch / 2)

## train set location
config.TRAIN.hr_img_path = 'DIV2K_train_HR/DIV2K_train_HR/'
config.TRAIN.lr_img_path = 'DIV2K_train_LR_bicubic_X4/DIV2K_train_LR_bicubic/X4/'

config.VALID = edict()
## test set location
config.VALID.hr_img_path = 'DIV2K_valid_HR/DIV2K_valid_HR/'
config.VALID.lr_img_path = 'DIV2K_valid_LR_bicubic_X4/DIV2K_valid_LR_bicubic/X4/'

def log_config(filename, cfg):
    with open(filename, 'w') as f:
        f.write("================================================\n")
        f.write(json.dumps(cfg, indent=4))
        f.write("\n================================================\n")

In [0]:
##importing tensorflow & tensorlayer
##scipy (based on numpy) for manipulating images -- downsampling and cropping etc.
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.prepro import *


import scipy
import numpy as np

def get_imgs_fn(file_name, path):
    """ Input an image path and name, return an image array """
    return scipy.misc.imread(path + file_name, mode='RGB')

def crop_sub_imgs_fn(x, is_random=True):
    x = crop(x, wrg=384, hrg=384, is_random=is_random)
    x = x / (255. / 2.)
    x = x - 1.
    return x

def downsample_fn(x):
    # We obtained the LR images by downsampling the HR images using bicubic kernel with downsampling factor r = 4.
    x = imresize(x, size=[96, 96], interp='bicubic', mode=None)
    x = x / (255. / 2.)
    x = x - 1.
    return x

In [0]:
t_image = tf.placeholder('float32', [64, 96, 96, 3], name='t_image_input_to_SRGAN_generator')
net_g = SRGAN_g(t_image, is_train=True, reuse=True)

[TL] InputLayer  SRGAN_g/in: (64, 96, 96, 3)


INFO:tensorlayer:InputLayer  SRGAN_g/in: (64, 96, 96, 3)


[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu


INFO:tensorlayer:Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu


[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation


...

INFO:tensorlayer:Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation


[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu


INFO:tensorlayer:SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu


[TL] Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


INFO:tensorlayer:Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


In [0]:
##making checkpoint directory in colab to upload d-train and g-train files
import os
os.mkdir("checkpoint")

In [0]:
#MAIN EXECUTION

import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy

#model.py consists of methods returning generator discriminator and vgg sample model
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
import tensorflow as tf
import tensorlayer as tl


## Adam optimizer user in this GAN
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = config.TRAIN.n_epoch_init
## adversarial learning (SRGAN)
n_epoch = config.TRAIN.n_epoch
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every

ni = int(np.sqrt(batch_size))


### ALL BASIC INIT BEFORE INIT AND TRAIN FROM BELOW SHIFTED HERE FOR SIMPLICITY

## create folders to save result images and trained model
save_dir_ginit = "samples/train_ginit"
save_dir_gan = "samples/train_gan"
tl.files.exists_or_mkdir(save_dir_ginit)
tl.files.exists_or_mkdir(save_dir_gan)
checkpoint_dir = "checkpoint"
tl.files.exists_or_mkdir(checkpoint_dir)

#loading images into the RAM using tensorflow -- takes up a lot of cloud memory
    #PRE-LOAD DATA
train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))
train_lr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.lr_img_path, regx='.*.png', printable=False))
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))

   
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)


    #DEFINE MODEL
#t_image is a tensorflow image variable with given params
t_image = tf.placeholder('float32', [batch_size, 96, 96, 3], name='t_image_input_to_SRGAN_generator')

#t_target_image is tensorflow variable for output (or) validation images
t_target_image = tf.placeholder('float32', [batch_size, 384, 384, 3], name='t_target_image')

#generator takes low-res image objects -- t_image
net_g = SRGAN_g(t_image, is_train=True, reuse=False)

#discriminator takes hi-res image objects from gen and dataset -- t_target_iamge
net_d, logits_real = SRGAN_d(t_target_image, is_train=True, reuse=False)
_, logits_fake = SRGAN_d(net_g.outputs, is_train=True, reuse=True)

#not mandatory -- logging about layers and parameters in the network
net_g.print_params(False)
net_g.print_layers()
net_d.print_params(False)
net_d.print_layers()


## vgg inference
t_target_image_224 = tf.image.resize_images(
    t_target_image, size=[224, 224], method=0,
    align_corners=False)  
t_predict_image_224 = tf.image.resize_images(net_g.outputs, size=[224, 224], method=0, align_corners=False)

net_vgg, vgg_target_emb = Vgg19_simple_api((t_target_image_224 + 1) / 2, reuse=False)
_, vgg_predict_emb = Vgg19_simple_api((t_predict_image_224 + 1) / 2, reuse=True)


#testing network object
net_g_test = SRGAN_g(t_image, is_train=False, reuse=True)


#defining training options

#discriminator loss for real and fake data (== logits)
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

#other losses
g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(net_g.outputs, t_target_image, is_mean=True)
vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)


##generator has only one loss -- sum of mean square error, total gan and vgg
g_loss = mse_loss + vgg_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

##learning rating
with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)
    ## Pretrain
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
    ## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

#restoring all the weights into the network (session) -- d-train g-train
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_train.npz', network=net_g) is False:
    tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_train_init.npz', network=net_g)
tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/d_train.npz', network=net_d)

#Loading vgg
vgg19_npy_path = "vgg19.npy"
npz = np.load(vgg19_npy_path, encoding='latin1').item()

params = []
for val in sorted(npz.items()):
    W = np.asarray(val[1][0])
    b = np.asarray(val[1][1])
    print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))
    params.extend([W, b])
tl.files.assign_params(sess, params, net_vgg)

##small test session before starting training
#Training # use first `batch_size` of train set to have a quick test during training
sample_imgs = train_hr_imgs[0:batch_size]
sample_imgs_384 = tl.prepro.threading_data(sample_imgs, fn=crop_sub_imgs_fn, is_random=False)
print('sample HR sub-image:', sample_imgs_384.shape, sample_imgs_384.min(), sample_imgs_384.max())
sample_imgs_96 = tl.prepro.threading_data(sample_imgs_384, fn=downsample_fn)
print('sample LR sub-image:', sample_imgs_96.shape, sample_imgs_96.min(), sample_imgs_96.max())
tl.vis.save_images(sample_imgs_96, [ni, ni], save_dir_ginit + '/_train_sample_96.png')
tl.vis.save_images(sample_imgs_384, [ni, ni], save_dir_ginit + '/_train_sample_384.png')
tl.vis.save_images(sample_imgs_96, [ni, ni], save_dir_gan + '/_train_sample_96.png')
tl.vis.save_images(sample_imgs_384, [ni, ni], save_dir_gan + '/_train_sample_384.png')



def train():

    #init generator -- needed only first time
    ###initGAN()

    #train GAN (SRGAN)
    trainGAN()

            
def initGAN():
    sess.run(tf.assign(lr_v, lr_init))
    print(" ** fixed learning rate: %f (for init G)" % lr_init)
    for epoch in range(0, n_epoch_init + 1):
        epoch_time = time.time()
        total_mse_loss, n_iter = 0, 0
        ## If your machine have enough memory, please pre-load the whole train set.
        for idx in range(0, len(train_hr_imgs), batch_size):
            step_time = time.time()
            b_imgs_384 = tl.prepro.threading_data(train_hr_imgs[idx:idx + batch_size], fn=crop_sub_imgs_fn, is_random=True)
            b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)
            ## update G
            errM, _ = sess.run([mse_loss, g_optim_init], {t_image: b_imgs_96, t_target_image: b_imgs_384})
            print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
            total_mse_loss += errM
            n_iter += 1
        log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
        print(log)

        ## quick evaluation on train set
        if (epoch != 0) and (epoch % 10 == 0):
            out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})  #; print('gen sub-image:', out.shape, out.min(), out.max())
            print("[*] save images")
            tl.vis.save_images(out, [ni, ni], save_dir_ginit + '/train_%d.png' % epoch)

        ## save model
        if (epoch != 0) and (epoch % 10 == 0):
            tl.files.save_npz(net_g.all_params, name=checkpoint_dir + '/g_train_init.npz', sess=sess)
  
  
def trainGAN():
    for epoch in range(0, n_epoch + 1):
        ## update learning rate
        if epoch != 0 and (epoch % decay_every == 0):
            new_lr_decay = lr_decay**(epoch // decay_every)
            sess.run(tf.assign(lr_v, lr_init * new_lr_decay))
            log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
            print(log)
        elif epoch == 0:
            sess.run(tf.assign(lr_v, lr_init))
            log = " ** init lr: %f  decay_every_init: %d, lr_decay: %f (for GAN)" % (lr_init, decay_every, lr_decay)
            print(log)

        epoch_time = time.time()
        total_d_loss, total_g_loss, n_iter = 0, 0, 0

        ##this preloads all the images into memory --  heavy usage of gpu
        for idx in range(0, len(train_hr_imgs), batch_size):
            step_time = time.time()
            b_imgs_384 = tl.prepro.threading_data(train_hr_imgs[idx:idx + batch_size], fn=crop_sub_imgs_fn, is_random=True)
            b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)
            ## update D
            errD, _ = sess.run([d_loss, d_optim], {t_image: b_imgs_96, t_target_image: b_imgs_384})
            ## update G
            errG, errM, errV, errA, _ = sess.run([g_loss, mse_loss, vgg_loss, g_gan_loss, g_optim], {t_image: b_imgs_96, t_target_image: b_imgs_384})
            print("Epoch [%2d/%2d] %4d time: %4.4fs, d_loss: %.8f g_loss: %.8f (mse: %.6f vgg: %.6f adv: %.6f)" %
                  (epoch, n_epoch, n_iter, time.time() - step_time, errD, errG, errM, errV, errA))
            total_d_loss += errD
            total_g_loss += errG
            n_iter += 1

        log = "[*] Epoch: [%2d/%2d] time: %4.4fs, d_loss: %.8f g_loss: %.8f" % (epoch, n_epoch, time.time() - epoch_time, total_d_loss / n_iter,
                                                                                total_g_loss / n_iter)
        print(log)

        ## quick evaluation on train set
        if (epoch != 0) and (epoch % 10 == 0):
            out = sess.run(net_g_test.outputs, {t_image: sample_imgs_96})
            print("[*] save images")
            tl.vis.save_images(out, [ni, ni], save_dir_gan + '/train_%d.png' % epoch)

        ## save model
        if (epoch != 0) and (epoch % 10 == 0):
            tl.files.save_npz(net_g.all_params, name=checkpoint_dir + '/g_train.npz', sess=sess)
            tl.files.save_npz(net_d.all_params, name=checkpoint_dir + '/d_train.npz', sess=sess)

            
#invoke above function
train()


#downloading generator and discrminiator weights after completion
from google.colab import files

files.download('checkpoint/g_train.npz')
files.download('checkpoint/d_train.npz')

[TL] [!] samples/train_ginit exists ...
[TL] [!] samples/train_gan exists ...
[TL] [!] checkpoint exists ...
[TL] read 32 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 64 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 96 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 128 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 160 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 192 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 224 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 256 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 288 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 320 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 352 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 384 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 416 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 448 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 480 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 512 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 544 from DIV2K_train_HR/DIV2K_train_HR/
[TL] read 576 from DIV2K_train_HR/DIV2K_trai

MessageError: ignored

# Evaluation

In [0]:
import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
import tensorflow as tf
import tensorlayer as tl
from tensorlayer.prepro import *
def evaluate():
    #create folders to save result images
    save_dir = "samples/evaluate"
    tl.files.exists_or_mkdir(save_dir)
    checkpoint_dir = "checkpoint"

    #model
    valid_lr_img = get_imgs_fn('var.jpg', 'sample_data/')
    valid_lr_img = (valid_lr_img / 127.5) - 1  # rescale to ［－1, 1]
    
    #creating tensorflow variable for images
    size = valid_lr_img.shape
    t_image = tf.placeholder('float32', [1, None, None, 3], name='input_image')

    ##initializing generator model
    net_g = SRGAN_g(t_image, is_train=False, reuse=True)

    #Restore model -- loading only g-train
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
    tl.layers.initialize_global_variables(sess)
    tl.files.load_and_assign_npz(sess=sess, name='g_srgan.npz', network=net_g)

    #eval starts here
    start_time = time.time()
    out = sess.run(net_g.outputs, {t_image: [valid_lr_img]})
    print("took: %4.4fs" % (time.time() - start_time))

    print("LR size: %s /  generated HR size: %s" % (size, out.shape))  # LR size: (339, 510, 3) /  gen HR size: (1, 1356, 2040, 3)
    print("[*] save images")
    
    #save generated and low res image
    tl.vis.save_image(out[0], save_dir + '/valid_gen.jpg')
    tl.vis.save_image(valid_lr_img, save_dir + '/valid_lr.jpg')

    #save enhanced image by bicubic interpolation
    out_bicu = scipy.misc.imresize(valid_lr_img, [size[0] * 4, size[1] * 4], interp='bicubic', mode=None)
    tl.vis.save_image(out_bicu, save_dir + '/valid_bicubic.jpg')
evaluate()

[TL] [!] samples/evaluate exists ...


INFO:tensorlayer:[!] samples/evaluate exists ...


[TL] InputLayer  SRGAN_g/in: (1, ?, ?, 3)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  # This is added back by InteractiveShellApp.init_path()
INFO:tensorlayer:InputLayer  SRGAN_g/in: (1, ?, ?, 3)


[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu


...

INFO:tensorlayer:Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


[TL] [*] Load g_srgan.npz SUCCESS!


INFO:tensorlayer:[*] Load g_srgan.npz SUCCESS!


took: 1.2060s
LR size: (720, 480, 3) /  generated HR size: (1, 2880, 1920, 3)
[*] save images


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:45: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
